In [1]:
import requests
from pprint import pprint
import pandas as pd
import json
from gensim import corpora, models, similarities

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
url = ("http://localhost:8082/api/tags/populated");
response = requests.get(url)
data = json.loads(response.text)
pprint(data)

ConnectionError: HTTPConnectionPool(host='localhost', port=8082): Max retries exceeded with url: /api/tags/populated (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10709d2e8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [ ]:
class TagCorpus(object):
    def __iter__(self):
        for tag in data:
            words = ""
            for chunk in tag['_chunks']:
                words += " " + str(chunk['content']);
            yield words.split()

In [ ]:
corpus = TagCorpus() # doesn't load the corpus into memory!
print(corpus)

In [ ]:
for vector in corpus: # load one vector into memory at a time
    print(vector)

In [ ]:
stoplist = set('for a of the and to in'.split())
# collect statistics about all tokens
dictionary = corpora.Dictionary(vector for vector in corpus)
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]

# dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids)

dictionary.compactify() # remove gaps in id sequence after words that were removed
print(dictionary)